**Implementación de una Máquina de Estados Finito (FSM) Interactiva**

Este programa implementa un reconocedor de entradas de un usuario basado en una FSM predefinida. Requiere que el diseñador especifique la *tabla de transición*.

Primero, necesitamos instalar algunos paquetes:

In [65]:
%pip install spacy
%python -m spacy download es_core_news_sm

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


Importamos algunas bibliotecas que necesitaremos:

In [66]:
import re
import es_core_news_sm
from string import punctuation
from unidecode import unidecode
from nltk.stem import SnowballStemmer
from spacy.lang.es.stop_words import STOP_WORDS

In [67]:

nlp = es_core_news_sm.load() # Cargamos modelo de lenguaje en Español
stemmer   = SnowballStemmer('spanish')  # Modelo del stemmer en español
_ERROR = (-1) # Estado de error
Num_Q = 4 # Numero de estados
# Vocabulario (alfabeto) de la máquina:
Sigma = {'internet', 'teléfono', 'tarjeta', 'mensual', 'casa', 'departamento'}
q0 = 0    # Estado inicial
F = {3}   # Estados finales


In [68]:
def Tokenizar(oracion):
    doc = nlp(oracion)
    tokens = [palabra.text for palabra in doc]
    return(tokens)

In [69]:
# @IN array oracion
# @OUT String Lemas (concatenados)
def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))

In [70]:
def EliminaPuntuacionAcento(texto):
    oracionsinacentos = unidecode(texto)
    string_numeros = re.sub(r'[\”\“\¿\°]','', oracionsinacentos)
    return ''.join(c for c in string_numeros if c not in punctuation)

In [71]:
def Reducir(oracion): 
   tokens = Tokenizar(oracion)
   stems = [stemmer.stem(palabra) for palabra in tokens]
   return(" ".join(stems))

In [72]:
#limpieza stopwords y puntuacion (aún no se lematiza ni se reduce (stemming))    
#@IN String oraciones
#@OUT list texto_limpio (lista de oraciones limpias)
def PreProcesamiento(oraciones):
   texto = oraciones.lower()
   texto = EliminaPuntuacionAcento(texto) 
   return (texto)


In [81]:
#Ahora, definimos el método **LeerRespuesta(pregunta, patron)**, que lee una respuesta hablada del usuario a partir de 
#una **pregunta** dada por el FA. En el caso de que la respuesta "calce" con algunos de los símbolos permitidos en **patron** 
#(i.e., símbolos de *Sigma*), se retorna el calce:
# @IN string pregunta
# @IN string patron
# @OUT string calce (solo primera coincidencia) o "nada"
def LeerRespuesta(pregunta, patron):
   
   texto = input(pregunta)
   # Probar que la frase contenga alguno de los símbolos del patron
   textoLimpio = PreProcesamiento(texto)
   lemas = Lematizar(textoLimpio)
   reducir = Reducir(lemas)
   print(reducir)
   calce = re.search(patron, reducir)
   if calce != None:
      return(calce.group(0))
   else:
      return("nada")
   
   

Luego, definimos el reconocedor basado en el FSM con la función **FSM(q0,F,Sigma,Preguntas,TablaTrans)**. Esta recibe el estado inicial (**Q0**) y final (**F**), el vocabulario **Sigma**, la lista de **Preguntas** para cada estado, y la tabla de transición **TablaTrans** que el FSM debe recorrer. La función retorna *True* si la entrada completa se reconoce, o *False*, en caso contrario.

Recuerde que el lenguaje a reconocer es el dado por el siguiente autómata:

<img src="https://drive.google.com/uc?id=1C-0X7inkaOesCbNqZFbYb50qY3ZNF38k" width="500" >

El algoritmo que implementa un FSM, dada una tabla de transición, es el siguiente:

* *q = estado inicial*
* *F = conjunto de estados finales (podría haber más de uno)*
* *Repita mientras q no está en F  o no sea un estado de error:*
   * *Simbolo = Entrada leida del usuario*
   * *(Actualizar el estado)*
   * *q = TablaTransicion[q][Simbolo]*
   * *(Validar que el simbolo exista para el estado actual)*
* *Fin repetición*
* *Si q está en F Entonces retorne TRUE (reconoce)*
* *Sino retorne FALSE (no reconoce)*


In [74]:
def FSM(Q0, F, Sigma, Preguntas, TablaTrans):
    q = Q0    # Estado inicial
    patron = "|".join(Sigma)
    # Repita mientras no sea un estado final o error
    while ( not(q in F)   and  (q != _ERROR)):
        Simbolo  = LeerRespuesta(Preguntas[q], patron)
        # Verifique que exista el índice "Simbolo"
        try:
           TablaTrans[q][Simbolo]
        except KeyError:
           q = _ERROR
           break
        # Asigne el siguiente estado de la tabla o bien ERROR si no existe
        q = (_ERROR if (not(Simbolo in Sigma)) else TablaTrans[q][Simbolo])
    return(q in F)

Necesitamos, una función que inicialice y defina los estados del FA, por lo que definimos la función **InicializarFA(nQ,Sigma)**, que toma el número de estados **nQ** de la máquina y el vocabulario **Sigma**, e inicializa los estados de la tabla de transición, según el grafo de interacción especificado en clases. La función retorna la tabla de transición llena (por defecto, los estados inválidos se llenan con un valor de **ERROR**).

Note que la matriz (tabla) de transición (*tt*) la podemos acceder por *fila* y *columna*, donde *fila* es el índice de un estado y *columna* es un símbolo del alfabeto **Sigma** (esto evita recordar la posición de los símbolos):

In [75]:
def InicializarFSM(nQ, Sigma):
   tt = {}
   # Crear tabla de transición vacia (con valor de error)
   for numQ in range (nQ):
      tt[numQ] = {}
      for Simb in Sigma:
         tt[numQ][Simb] = _ERROR
   # LLenar transiciones no vacias
   tt[0]['internet']        = 1
   tt[0]['teléfono']        = 2
   tt[1]['tarjeta']         = 1
   tt[1]['mensual']         = 3
   tt[2]['casa']            = 3
   tt[2]['departamento']    = 1
   return(tt)

Ahora definimos la función **EspecificarPreguntas()**, que simplemente especifica las preguntas que el FA realizará al usuario en cada estado, retornando la lista de preguntas posibles (existe una pregunta por cada estado, exceptuando el final):

In [76]:
def EspecificarPreguntas():
   Preguntas = ["Qué servicio requiere? ",
                "¿Qué tipo de cliente? ",
                "¿En qué residencia? "
            ]
   return(Preguntas)

In [77]:
TablaTransicion = InicializarFSM(Num_Q, Sigma)
Preguntas  = EspecificarPreguntas()

Comenzamos la interacción con la FSM:

In [82]:
print("Bienvenido a MasterPlop Comunicaciones..")
status = FSM(q0, F, Sigma, Preguntas, TablaTransicion)
if (status):
    print("Transacción aceptada!")
else:
    print("Error, no se entendió la consulta!")

Bienvenido a MasterPlop Comunicaciones..
<re.Match object; span=(35, 43), match='internet'>
None
Error, no se entendió la consulta!
